In [19]:
from __future__ import annotations
from pprint import pformat
from typing import List, Set, Tuple

ROWS = COLS = 9
NUMBERS = [x for x in range(1, 9 + 1)]

def main():
    grid1 = [
        [0, 0, 4, 0, 0, 0, 1, 0, 0],
        [0, 0, 2, 0, 5, 0, 0, 7, 0],
        [8, 3, 6, 1, 0, 0, 0, 0, 4],
        [9, 0, 8, 0, 0, 0, 4, 3, 1],
        [0, 1, 7, 0, 0, 4, 8, 0, 0],
        [0, 2, 5, 0, 3, 0, 7, 6, 9],
        [2, 0, 9, 6, 0, 0, 5, 1, 7],
        [6, 7, 3, 0, 1, 0, 2, 0, 8],
        [5, 8, 0, 2, 4, 7, 6, 9, 3],
    ]

    grid = Grid(grid1)
    results = solve_all(grid)
    for r in results:
        print(r)
class Grid:

    _values: List[List[int]]

    def __init__(self, values: List[List[int]]):
        assert isinstance(values, list)
        assert len(values) == ROWS
        for row in values:
            assert isinstance(row, list)
            assert len(row) == COLS
        self._values = values
    def __hash__(self):
        return hash(''.join(str(x) for row in self._values for x in row))

    def __str__(self):
        return '{}(\n{}\n)'.format(type(self).__name__, pformat(self._values))

    def solved(self) -> bool:
        all_values = [x for row in self._values for x in row]
        return 0 not in all_values

    def possible_numbers(self) -> List[Tuple[int, int, List[int]]]:
        return [
            (row, col, self._possible_numbers_for_cell(row, col))
            for row, values in enumerate(self._values)
            for col, x in enumerate(values)
            if x == 0
        ]

    def clone_filled(self, row, col, number) -> Grid:
        values = [[x for x in row] for row in self._values]
        values[row][col] = number
        return type(self)(values)

    def _possible_numbers_for_cell(self, row, col) -> List[int]:
        row_numbers = [x for x in self._values[row]]
        col_numbers = [row[col] for row in self._values]
        block_numbers = self._block_numbers(row, col)

        return [
            x
            for x in NUMBERS
            if (x not in row_numbers)
            and (x not in col_numbers)
            and (x not in block_numbers)
        ]

    def _block_numbers(self, row, col) -> List[int]:
        row_start = (row // 3) * 3
        col_start = (col // 3) * 3
        return [
            x
            for row in self._values[row_start : row_start + 3]
            for x in row[col_start : col_start + 3]
        ]

def solve_all(grid: Grid) -> Set[Grid]:
    solutions = set()
    def _solve(grid: Grid):
        if grid.solved():
            solutions.add(grid)
            return
        possible_numbers = grid.possible_numbers()
        row, col, numbers = min(possible_numbers, key=lambda x: len(x[-1]))

        if not numbers:
            return
        for number in numbers:
            next_grid = grid.clone_filled(row, col, number)
            _solve(next_grid)

    _solve(grid)

    return solutions

if __name__ == '__main__':
    main()


Grid(
[[7, 5, 4, 3, 9, 6, 1, 8, 2],
 [1, 9, 2, 4, 5, 8, 3, 7, 6],
 [8, 3, 6, 1, 7, 2, 9, 5, 4],
 [9, 6, 8, 7, 2, 5, 4, 3, 1],
 [3, 1, 7, 9, 6, 4, 8, 2, 5],
 [4, 2, 5, 8, 3, 1, 7, 6, 9],
 [2, 4, 9, 6, 8, 3, 5, 1, 7],
 [6, 7, 3, 5, 1, 9, 2, 4, 8],
 [5, 8, 1, 2, 4, 7, 6, 9, 3]]
)
